# Plots of colored noise generation transform

In [ ]:
from torchsig.utils.dsp import (
    torchsig_float_data_type,
    torchsig_complex_data_type
)

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

In [ ]:
N = 100000
rng = np.random.default_rng(seed=42)
#noise = (1 + 1j) / np.sqrt(2)
noise = (rng.standard_normal((1,)) + 1j*rng.standard_normal((1,))) / np.sqrt(2)
power = np.abs(noise)**2
print(power)

noise0 = np.zeros((N,),dtype=torchsig_complex_data_type)
print(noise.dtype)
print(noise0.dtype)
noise0

In [ ]:
# noise generation utility
def noise_generator(
    N: int = 1024,
    power: float = 1.0,
    color: str = 'white',
    continous: bool = True,
    rng: np.random.Generator = np.random.default_rng(seed=None)
) -> np.ndarray:
    """Generates additive complex noise of specified power and type.

    Args:
        power (float): Desired noise power (linear, positive). Defaults to 1.0 W (0 dBW).
        color (str): Noise color, supports 'white', 'pink', or 'red' noise frequency spectrum types. Defaults to 'white'.
        continuous (bool): Sets noise to continuous (True) or impulsive (False). Defaults to True.
        rng (np.random.Generator, optional): Random number generator. Defaults to np.random.default_rng(seed=None).

    Raises:
        ValueError: If invalid noise power specified.
        ValueError: If unsupported noise type specified.
    
    Returns:
        np.ndarray: Complex noise samples with specified power.
    
    """
    if not power >= 0.:
         raise ValueError(f"Noise power must be greater than or equal to 0.")
    
    # x_white = (rng.standard_normal((N,)) + 1j*rng.standard_normal((N,))) / np.sqrt(2) # white noise source: 1.0 W (0 dBW)
    # X_white = np.fft.fft(x_white, norm="ortho") # white noise 1.0 W, frequency domain
    # X_white = np.fft.rfft(np.random.randn(N), norm="ortho")

    if continous:
        noise_source = (   rng.standard_normal((N,), dtype=torchsig_float_data_type) + 
                        1j*rng.standard_normal((N,), dtype=torchsig_float_data_type)) / np.sqrt(2) # continous white noise (1.0 W)
    else: # impulsive
        noise_source = np.zeros((N,), dtype=torchsig_complex_data_type)
        impulse_ind = rng.integers(0,N,dtype=int)   # random impulse location
        noise_source[impulse_ind] = (1 + 1j) / np.sqrt(2) # impulse 1.0 W

    X_white = np.fft.fft(noise_source, norm="ortho") # frequency domain noise 1.0 W
    
    #complex-valued samples
    # print("x_white (time domain) energy: ", np.sum(np.abs(x_white)**2))
    # print("x_white (time domain) power: ", np.sum(np.abs(x_white)**2)/len(x_white))
    #print("X_white (frequency domain) power: ", np.sum(np.abs(X_white*np.conj(X_white)))/len(X_white))
    
    # frequency domain shaping filter
    #freqs = np.fft.rfftfreq(N) # sample frequencies 
    freqs = np.fft.fftfreq(N) # sample frequencies 
    
    if color == 'white': # flat frequency spectrum
        S = 1
        #S = np.ones((N,))
        #S = S / np.sqrt(np.mean(S**2)) # RMS normalize shaping filter (estimated)
    elif color == 'pink': # 1/f (flicker noise), -10 db/decade frequency power spectrum
        S = 1/np.where(freqs == 0, float('inf'), np.sqrt(np.abs(freqs))) # zero-mean (DC=0) 
        S = S / np.sqrt(np.mean(S**2)) # RMS normalize shaping filter (estimated)
    elif color == 'red': # 1/f**2 (brownian noise), -20 dB/decade frequency power spectrum
        S = 1/np.where(freqs == 0, float('inf'), np.abs(freqs)) # zero-mean (DC=0)
        S = S / np.sqrt(np.mean(S**2)) # RMS normalize shaping filter (estimated)
    else:
        raise ValueError(f"Invalid noise type {type}. Must be 'white', 'pink', or 'red'.")
    
    X_shaped = S * X_white 
    noise = np.fft.ifft(X_shaped, norm="ortho")
    est_power = np.sum(np.abs(noise)**2)/len(noise)
    noise = np.sqrt(power / est_power) * noise 

    print(len(X_white))
    print("X_white (frequency domain) power: ", np.sum(np.abs(X_white)**2)/len(X_white))
    #if isinstance(S,np.ndarray): print("S shaping: ", np.sum(np.abs(S)**2)/len(S))
    print("X_shaped (frequency domain) power: ", np.sum(np.abs(X_shaped)**2)/len(X_shaped))
    print("noise (time domain) output power: ", np.sum(np.abs(noise)**2)/len(noise))
    print("Min,Mean,Max: ",np.min(np.abs(noise)),np.mean(np.abs(noise)),np.max(np.abs(noise)))
    
    return noise

In [ ]:
# functional
def additive_noise(
    data: np.ndarray,
    power: float = 1.0,
    color: str = 'white',
    continuous: bool = True,
    rng: np.random.Generator = np.random.default_rng(seed=None)
) -> np.ndarray:
    """Additive complex noise of specified power and type.

    Args:
        data (np.ndarray): Complex valued IQ data samples.
        power (float): Desired noise power (linear, positive). Defaults to 1.0 W (0 dBW).
        color (str): Noise color, supports 'white', 'pink', or 'red' noise frequency spectrum types. Defaults to 'white'.
        continuous (bool): Sets noise to continuous (True) or impulsive (False). Defaults to True.
        rng (np.random.Generator, optional): Random number generator. Defaults to np.random.default_rng(seed=None).
    
    Returns:
        np.ndarray: Data with complex noise samples with specified power added.
    
    """
    N = data.size
    noise_samples = noise_generator(N, power, color, continuous, rng)
    return (data + noise_samples).astype(torchsig_complex_data_type)

In [ ]:
# continous noise generator plots
N = 100000
freqs = np.fft.fftfreq(N)

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

noise_white = noise_generator(N, 1.0, 'white', True, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_white, Fs=1.0, scale='linear', sides='twosided', color='white');

noise_pink = noise_generator(N, 1.0, 'pink', True, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_pink, Fs=1.0, scale='linear', sides='twosided', color='pink');

noise_red = noise_generator(N, 1.0, 'red', True, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_red, Fs=1.0, scale='linear', sides='twosided', color='red');

ax.set_yscale('log')
# ax.set_xscale('log')
ax.set_xscale('symlog', 
             base=10,          # Logarithm base
             linthresh=1/N)   # Threshold for linear region near zero

# Configure axis labels and grid
ax.set_xlabel('Frequency (Hz) [log]');
ax.set_ylabel('Magnitude [log]');
#plt.ylim([1e-5, None]);

In [ ]:
# impulsive noise generator plots
N = 100000
freqs = np.fft.fftfreq(N)

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

noise_white = noise_generator(N, 1.0, 'white', False, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_white, Fs=1.0, scale='linear', sides='twosided', color='white');

noise_pink = noise_generator(N, 1.0, 'pink', False, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_pink, Fs=1.0, scale='linear', sides='twosided', color='pink');

noise_red = noise_generator(N, 1.0, 'red', False, np.random.default_rng(42))
spectrum, freqs, _ = ax.magnitude_spectrum(noise_red, Fs=1.0, scale='linear', sides='twosided', color='red');

ax.set_yscale('log')
# ax.set_xscale('log')
ax.set_xscale('symlog', 
             base=10,          # Logarithm base
             linthresh=1/N)   # Threshold for linear region near zero

# Configure axis labels and grid
ax.set_xlabel('Frequency (Hz) [log]');
ax.set_ylabel('Magnitude [log]');
#plt.ylim([1e-5, None]);

In [ ]:
# test functional additive noise

# continous noise generator plots
N = 100000
data_z = np.zeros((N,), dtype=torchsig_complex_data_type)
freqs = np.fft.fftfreq(N)

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

noise_z = additive_noise(data_z, 1.0, 'white', True, np.random.default_rng(42))
# spectrum, freqs, _ = ax.magnitude_spectrum(noise_z, Fs=1.0, scale='linear', sides='twosided', color='white');

# ax.set_yscale('log')
# # ax.set_xscale('log')
# ax.set_xscale('symlog', 
#              base=10,          # Logarithm base
#              linthresh=1/N)   # Threshold for linear region near zero

# # Configure axis labels and grid
# ax.set_xlabel('Frequency (Hz) [log]');
# ax.set_ylabel('Magnitude [log]');
# #plt.ylim([1e-5, None]);

print(noise_z)